# 2D Trapezoidal Integrator

In [82]:
import numpy as np

Here, we compute the composite trapezoidal rule in two variables. There are broadly two steps to our procedure. First, we compute $$(1) \ \ \ \  F(x_i) = \int_{g(x_i)}^{h(x_i)}f(x_i,y)dy.$$ Then, we 'integrate' (ie apply our weighted sums) to the resultant $F$ values over $x$, via $$(2) \ \ \ \ \int\int_{\Omega} f(x,y)dydx = \int_a^bF(x)dx.$$

There are two options for the method below: One that fills out a grid, and another that sets $n^y_i$ based on the size of $[g_i,h_i]$.

Grid method: We make the decision to fix $n^y_i$ across $x_i$ values. That is, for step (1) above, rather than change the number of subintervals as the interval $[g(x_i),h(x_i)]$ changes, we will simply change the size of these intervals. This may have a strange effect on the convergence / error bounds, but it will simplify our computations a great deal. We store this variable as 'ny' below. Indeed, we intend to iterate in an unevenly spaced grid, so we will have n = nx = ny.

We will proceed with our attempted convergence algorithm by doubling n at each step. Observe that as we do this, we effectively square the number of points at which we interpolate. In our previous 1-D trapezoidal integrator, storing old function values cut down on total function evaluations by 50%, but, because of this squaring, keeping old evaluations only cuts down on total function evaluations by 25%. 

We also run into another problem: boundary points of the domain may not be amenable to our dividing method. Consider the right quarter circle; for $x_i=1$, we can have at most $ny=1$. Our solution could be to avoid splitting up the boundary. However, in the case below, we allow our integrator to attempt to divide the integral [1,1] into n subintervals at each step, observing that this should produce $F(1) \approx 0$ as desired. 

In [84]:
def compute_two_dim_trapezoidal_integrator_grid(a,b,getF,gfunc,hfunc,maxit = 10,tol=1.e-6):
    Function_Evals = 0
    n = 1
    Approximations = []

    fsLists = None
    it=0


    def get_F_xi(xi,ny,fsPrev = None):
        nonlocal getF,gfunc,hfunc,Function_Evals
        f = getF(xi)
        if fsPrev is None:
            # if we've never visited this xi before, we need to add all its nodes at the same time
            ai,bi = gfunc(xi),hfunc(xi)
            h = (bi-ai)/ny

            ys = np.linspace(ai,bi,ny+1)
            Function_Evals += len(ys)
            fs = f(ys)
            return(np.sum(np.array(fs)*h)-(fs[0]+fs[-1])*h/2,fs)
        else:
            fsPrev = np.array(fsPrev)
            # otherwise, we add only the odd nodes; that is, we add 2^(k-1) points spaced out by 2h

            ai,bi = gfunc(xi),hfunc(xi)

            h=(bi-ai)/ny
            ys = np.linspace(ai+h,bi-h,int(ny/2))
            Function_Evals += len(ys)
            newFs = np.empty(ny+1,dtype=float)
            newFs[::2] = fsPrev
            newFs[1::2] = f(ys)

            return(np.sum(np.array(newFs)*h)-(newFs[0]+newFs[-1])*h/2,newFs)
    
    def getInt(nx,Fs):
        nonlocal a,b
        h = (b-a)/nx
        return(np.sum(np.array(Fs)*h)-(Fs[0]+Fs[-1])*h/2)

    def full_procedure(n,fsPrev_lists):

        nonlocal a,b
        h=(b-a)/n
        nOld = n/2
        oldxis= np.linspace(a,b,int(nOld)+1)
        fsNew_lists = []
        Fxis = []
        
        # for each previously visited xi, we interpolate at the midpoints of each interval and generate a new F(xi) accordigly
        if fsPrev_lists is not None:
            for idx,l in enumerate(fsPrev_lists):
                xidx = oldxis[idx]
                Fxi,fsNew = get_F_xi(xidx,n,l)
                fsNew_lists.append(fsNew)
                Fxis.append(Fxi)


        # for our unvisited xis, we generate a full F(xi) approximation with n interpolation points
        if n > 1:
            newxis = np.linspace(a+h,b-h,int(nOld))
        else:
            newxis = np.array([a,b])
        fsNew_newPoint_lists = []
        Fxis_newPoints = []
        for xi in newxis:
            Fxi,fsNew = get_F_xi(xi,n,None)
            fsNew_newPoint_lists.append(fsNew)
            Fxis_newPoints.append(Fxi)

        # easier to not with numpy arrays here
        if fsPrev_lists is not None:
            fs_lists = [None] * (len(fsNew_lists) + len(fsNew_newPoint_lists))  
            Fxis_list = np.empty(len(fs_lists), dtype=float)

            # insert our new lists into our old lists
            fs_lists[::2] = fsNew_lists
            fs_lists[1::2] = fsNew_newPoint_lists

            Fxis_list[::2] = Fxis
            Fxis_list[1::2] = Fxis_newPoints

        else:
            # unless we're at step 1, in which case our lists are only our new lists
            fs_lists = fsNew_newPoint_lists
            Fxis_list = Fxis_newPoints

        # now integrate wrt x
        integral = getInt(n,Fxis_list)

        return(integral,fs_lists)
    
    absDif = None
    while ((len(Approximations) < 2) or (True if absDif is None else absDif > tol)) and (it < maxit):
        it += 1
        print(f"\nat iterate {it} out of {maxit}; n = {n}; evals = {Function_Evals}; absolute difference between curr and prev eval = {absDif}")
        # print(f"evals = {Function_Evals}")
        # print(f"absolute dif between curr and prev eval = {absDif}")
        integ,curr_fsLists = full_procedure(n,fsLists)
        fsLists = curr_fsLists
        Approximations.append(integ)
        if len(Approximations) >1:
            absDif = np.abs(Approximations[-1]-Approximations[-2])
        n = 2*n
    if it < maxit:
        print(f"\nbeat tolerance with absolute dif of {absDif}\n\n")

    return Approximations,Function_Evals

Variable method: Here, we adjust $n^y_i$ based on the length of the interval $[g(x_i),h(x_i)]$. We would like to preserve the 'power of two' injection we were doing before that allowed us to place our new interpolation points at the odd incides of our next array. Hence, we will initialize each new $n_y^i$ as $$n^y_{i,0} = \max\{1,n_x 2^{\lceil \log_2(\frac{h_i-g_i}{b-a})\rceil}\}    .$$ Then, at each step, we will double each $n^y_i$ as before, so $n_{i,k}^y = 2n_{i,k-1}^y$. In this way, we initialize our $n_i^y$ in a way that is at least somewhat proportional to the relative lengths of $[g_i,h_i]$ and $[a,b]$. 

We will handle each 0-length interval by forcing its $n^y_i=0$ at each step and then setting its $F$ value to 0. 

In [ ]:
def compute_two_dim_trapezoidal_integrator_variable(a,b,getF,gfunc,hfunc,maxit = 10,tol=1.e-6):
    Function_Evals = 0
    Approximations = []
    it=0


    def get_F_xi(xi,ny,fsPrev = None):
        nonlocal getF,gfunc,hfunc,Function_Evals
        f = getF(xi)
        if ny == 0:
            # if our interval is small enough, ignore this interpolation point (no volume to integrate over)
            return(0,0)
        if fsPrev is None:
            # if we've never visited this xi before, we need to add all its nodes at the same time
            ny=int(ny)
            ai,bi = gfunc(xi),hfunc(xi)
            h = (bi-ai)/ny

            ys = np.linspace(ai,bi,ny+1)
            Function_Evals += len(ys)
            fs = f(ys)
            return(np.sum(np.array(fs)*h)-(fs[0]+fs[-1])*h/2,fs)
        else:
            fsPrev = np.array(fsPrev)
            ny=int(ny)
            # otherwise, we add only the odd nodes; that is, we add 2^(k-1) points spaced out by 2h

            ai,bi = gfunc(xi),hfunc(xi)

            h=(bi-ai)/ny
            ys = np.linspace(ai+h,bi-h,int(ny/2))
            Function_Evals += len(ys)
            newFs = np.empty(ny+1,dtype=float)
            newFs[::2] = fsPrev
            newFs[1::2] = f(ys)

            return(np.sum(np.array(newFs)*h)-(newFs[0]+newFs[-1])*h/2,newFs)
    
    def getInt(nx,Fs):
        nonlocal a,b
        h = (b-a)/nx
        return(np.sum(np.array(Fs)*h)-(Fs[0]+Fs[-1])*h/2)

    def full_procedure(nx,nys,fsPrev_lists):

        nonlocal a,b
        l = b-a
        hx=(b-a)/nx
        nxOld = nx/2

        oldxis= np.linspace(a,b,int(nxOld)+1)

        fsNew_lists = []
        Fxis = []

        # for each previously visited xi, we interpolate at the midpoints of each interval and generate a new F(xi) accordigly
        if fsPrev_lists is not None:
            for idx,flist in enumerate(fsPrev_lists):
                xidx = oldxis[idx]
                nys[idx] = nys[idx]*2
                Fxi,fsNew = get_F_xi(xidx,nys[idx],flist)
                fsNew_lists.append(fsNew)
                Fxis.append(Fxi)


        # for our unvisited xis, we generate a full F(xi) approximation with n interpolation points
        if nx > 1:
            newxis = np.linspace(a+hx,b-hx,int(nxOld))
        else:
            newxis = np.array([a,b])
            ## here, we create our array of nys if uninitizlied
            nysNew = [0 if np.abs(gfunc(a)-hfunc(a)) < 1.e-6 else 1,0 if np.abs(gfunc(b)-hfunc(b)) < 1.e-6 else 1]


        fsNew_newPoint_lists = []
        Fxis_newPoints = []

        # here, we initialize our new nys
        if nx > 1:
            nysNew = []
            for xi in newxis:
                li = hfunc(xi)-gfunc(xi)
                # print(f"xi = {xi}")
                if np.abs(li) < 1.e-6:
                    nysNew.append(0)
                else:

                    frac = li/l
                    # print(f"frac = {frac}")
                    logarithm = np.log2(nx * li/l)
                    check = 2**np.ceil(logarithm)
                    # print(f"logarithm = {logarithm}")
                    nysNew.append(max(check,1))


        # evaluate at each of the new xis according to our nys
        for idx,xi in enumerate(newxis):
            Fxi,fsNew = get_F_xi(xi,nysNew[idx],None)
            fsNew_newPoint_lists.append(fsNew)
            Fxis_newPoints.append(Fxi)


        # easier to not work with numpy arrays here
        if fsPrev_lists is not None:
            fs_lists = [None] * (len(fsNew_lists) + len(fsNew_newPoint_lists))  
            Fxis_list = np.empty(len(fs_lists), dtype=float)
            nys_list = np.empty(len(fs_lists),dtype = float)

            # insert our new lists into our old lists
            fs_lists[::2] = fsNew_lists
            fs_lists[1::2] = fsNew_newPoint_lists

            Fxis_list[::2] = Fxis
            Fxis_list[1::2] = Fxis_newPoints

            nys_list[::2] = nys
            nys_list[1::2] = nysNew


        else:
            # unless we're at step 1, in which case our lists are only our new lists
            fs_lists = fsNew_newPoint_lists
            Fxis_list = Fxis_newPoints
            nys_list = nysNew

        # now integrate wrt x
        integral = getInt(nx,Fxis_list)

        return(integral,fs_lists,nys_list)
    
    absDif = None
    if it == 0:
        nys = []
        nx = 1
        fsLists = None
    while ((len(Approximations) < 2) or (True if absDif is None else absDif > tol)) and (it < maxit):
        it += 1
        print(f"\nAt iterate {it} out of {maxit}; nx = {nx}; evals = {Function_Evals}; absolute difference between curr and prev eval = {absDif}")
        # print(f"evals = {Function_Evals}")
        # print(f"absolute dif between curr and prev eval = {absDif}")
        integ,fsLists,nys= full_procedure(nx,nys,fsLists)
        Approximations.append(integ)
        if len(Approximations) >1:
            absDif = np.abs(Approximations[-1]-Approximations[-2])
        nx = 2*nx
    if it < maxit:
        print(f"\nbeat tolerance with absolute dif of {absDif}\n\n")

    return Approximations,Function_Evals

$e^{-xy}$ is a nice function to integrate because its integral has no analytical solution. 

In [ ]:
# we will generate exponentials for fixed x values
def getExp_x_i(xi):
    def f(y):
        return(np.exp(-xi*y))
    return(f)

In [86]:
def g0(x):
    return(0)
def hcircle(x):
    return(np.sqrt((1-x**2)))
def h1(x):
    return(1)


Here, we integrate our function $e^{-xy}$ over $[0,1] \times [0,1]$ and $\{(x,y) : x^2 + y^2 \leq 1 \ \mathrm{ and } \  x,y \geq 0\}$.

In [ ]:
# integrate over the square with the grid method
print(f"integrating over the square")
approxs,evals = compute_two_dim_trapezoidal_integrator_grid(0,1,getExp_x_i,gfunc=g0,hfunc=h1,maxit=25,tol=1.e-6)
print(f"our approximations were: ")
print(np.array(approxs))

integrating over the square

at iterate 1 out of 25; n = 1; evals = 0; absolute difference between curr and prev eval = None

at iterate 2 out of 25; n = 2; evals = 4; absolute difference between curr and prev eval = None

at iterate 3 out of 25; n = 4; evals = 9; absolute difference between curr and prev eval = 0.03514453452363564

at iterate 4 out of 25; n = 8; evals = 25; absolute difference between curr and prev eval = 0.0077443804226215995

at iterate 5 out of 25; n = 16; evals = 81; absolute difference between curr and prev eval = 0.0018657894581991519

at iterate 6 out of 25; n = 32; evals = 289; absolute difference between curr and prev eval = 0.00046196757014604906

at iterate 7 out of 25; n = 64; evals = 1089; absolute difference between curr and prev eval = 0.00011521054549112897

at iterate 8 out of 25; n = 128; evals = 4225; absolute difference between curr and prev eval = 2.8785030848577087e-05

at iterate 9 out of 25; n = 256; evals = 16641; absolute difference between c

In [ ]:
# integrate over the quarter circle with the grid method
print(f"integrating over the quarter circle")
approxs,evals = compute_two_dim_trapezoidal_integrator_grid(0,1,getExp_x_i,gfunc=g0,hfunc=hcircle,maxit=25,tol=1.e-6)
print(f"our approximations were: ")
print(np.array(approxs))

integrating over the quarter circle

at iterate 1 out of 25; n = 1; evals = 0; absolute difference between curr and prev eval = None

at iterate 2 out of 25; n = 2; evals = 4; absolute difference between curr and prev eval = None

at iterate 3 out of 25; n = 4; evals = 9; absolute difference between curr and prev eval = 0.1028195175085449

at iterate 4 out of 25; n = 8; evals = 25; absolute difference between curr and prev eval = 0.04366267806082336

at iterate 5 out of 25; n = 16; evals = 81; absolute difference between curr and prev eval = 0.01772623711345378

at iterate 6 out of 25; n = 32; evals = 289; absolute difference between curr and prev eval = 0.00687576172132609

at iterate 7 out of 25; n = 64; evals = 1089; absolute difference between curr and prev eval = 0.002586766133238383

at iterate 8 out of 25; n = 128; evals = 4225; absolute difference between curr and prev eval = 0.0009539367444846292

at iterate 9 out of 25; n = 256; evals = 16641; absolute difference between curr

In [ ]:
# integrate over the square, variable method
print(f"integrating over the square")
approxs,evals = compute_two_dim_trapezoidal_integrator_variable(0,1,getExp_x_i,gfunc=g0,hfunc=h1,maxit=25,tol=1.e-6)
print(f"our approximations were: ")
print(np.array(approxs))

integrating over the square

At iterate 1 out of 25; nx = 1; evals = 0; absolute difference between curr and prev eval = None

At iterate 2 out of 25; nx = 2; evals = 4; absolute difference between curr and prev eval = None

At iterate 3 out of 25; nx = 4; evals = 9; absolute difference between curr and prev eval = 0.03514453452363564

At iterate 4 out of 25; nx = 8; evals = 25; absolute difference between curr and prev eval = 0.0077443804226215995

At iterate 5 out of 25; nx = 16; evals = 81; absolute difference between curr and prev eval = 0.0018657894581991519

At iterate 6 out of 25; nx = 32; evals = 289; absolute difference between curr and prev eval = 0.00046196757014604906

At iterate 7 out of 25; nx = 64; evals = 1089; absolute difference between curr and prev eval = 0.00011521054549112897

At iterate 8 out of 25; nx = 128; evals = 4225; absolute difference between curr and prev eval = 2.8785030848577087e-05

At iterate 9 out of 25; nx = 256; evals = 16641; absolute difference 

In [ ]:
# integrate over the quarter circle, variable method
print(f"integrating over the quarter circle")
approxs,evals = compute_two_dim_trapezoidal_integrator_variable(0,1,getExp_x_i,gfunc=g0,hfunc=hcircle,maxit=25,tol=1.e-6)
print(f"our approximations were: ")
print(np.array(approxs))

integrating over the quarter circle

At iterate 1 out of 25; nx = 1; evals = 0; absolute difference between curr and prev eval = None

At iterate 2 out of 25; nx = 2; evals = 2; absolute difference between curr and prev eval = None

At iterate 3 out of 25; nx = 4; evals = 6; absolute difference between curr and prev eval = 0.1028195175085449

At iterate 4 out of 25; nx = 8; evals = 20; absolute difference between curr and prev eval = 0.04366267806082336

At iterate 5 out of 25; nx = 16; evals = 68; absolute difference between curr and prev eval = 0.01776080785596812

At iterate 6 out of 25; nx = 32; evals = 256; absolute difference between curr and prev eval = 0.006847441816111077

At iterate 7 out of 25; nx = 64; evals = 984; absolute difference between curr and prev eval = 0.002582175559013522

At iterate 8 out of 25; nx = 128; evals = 3872; absolute difference between curr and prev eval = 0.0009526261691193971

At iterate 9 out of 25; nx = 256; evals = 15280; absolute difference bet

Observe that our variable method and grid method cost the same number of function evaluations over the square domain. This makes sense: at each new $x_i$ in the variable method, $l_i = l$, so $n^y_i = n_x$. On the other hand, over the circle, observe that (for the given convergence criteria) our variable method takes 15507200 evaluations while our grid method takes 16785409 evaluations. This is a difference of 16,785,409-15,507,200 $\approx$ 1.3e8 evaluations, or about 10%.  
